# Trabalho 1 - Machine Learning II 
Prof: Carlos Padilha

#### Alunos:  

Roberto A. Coutinho  
Thais Galho


## Sistemas com Multi-classificadores ou Ensembles

#### Este trabalho visa avaliar o entendimento em relaçãao á construção de sistemas com multi-classificadores ou ensembles. Para tal, os alunos deverão fazer o seguinte:


* Implementar o algoritmo AdaBoost (nos mesmos moldes que fizemos com o algoritmo Bagging).
    – Podem escolher qualquer tipo de classificador (MLP, SVM, etc).
* Processar os dados presente no arquivo sonar.all-data.
* Realizar treinamento e teste usando validação cruzada com 10 folds.
* Avaliar os resultados em termos de acurácia, recall e precisão.

Obs: O trabalho pode ser feito em dupla e deve ser enviado por email (carlos.engcomp@gmail.com).

In [1]:
import numpy as np
import pandas as pd

# Modelos
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# K-fold CrossValidation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
imported_data = pd.read_csv('sonar.all-data.csv', header=None)
imported_data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
# Separação entre dados e labels

labels = imported_data.iloc[:,-1]
data = imported_data.iloc[:,:-1]
len(data), len(labels)

(208, 208)

In [98]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score

def printCM(Y_test, predictions):
    cm = confusion_matrix(Y_test, predictions)
    acc_score = accuracy_score(Y_test, predictions)
    prec_score = precision_score(Y_test, predictions, average='weighted')
    
    print ('Confusion Matrix : ')
    print (cm)
    print
    
    tn = float(cm[0][0])
    fp = float(cm[0][1])
    fn = float(cm[1][0])
    tp = float(cm[1][1])

    actual_yes = fn+tp
    actual_no = tn+fp
    predicted_yes = fp+tp
    predicted_no = tn+fn

    total = float(len(imported_data))
    print ('Total : '+ str(total))

    print ('Acurácia : ' + str(acc_score))

    misclassification_rate = round((fp+fn)/total,3) # Overall, how often is it wrong?
    print ('Misclassification rate : ' +str(misclassification_rate))

    true_positive = round(tp/actual_yes,3) # When it's actually yes, how often does it predict yes?
    print ('True positives : ' +str(true_positive))

    false_positive = round(fp/actual_no,3) # When it's actually no, how often does it predict yes?
    print ('False positives : ' +str(false_positive))

    specificity = round(tn/actual_no,3) # When it's actually no, how often does it predict no?
    print ('Specificity : ' +str(specificity))

    #precision = round(tp/predicted_yes,3) # When it predicts yes, how often is it correct?
    print ('Precision : ' +str(prec_score))

    prevalence = round(actual_yes/total,3) # How often does the yes condition actually occur in our sample?
    print ('Prevalence : ' +str(prevalence))
    
    recall = round(tp / (tp + fn), 3)
    print ('Recall : ' +str(recall))

    #f1 = round(2 * ((precision * true_positive) / (precision + true_positive)),3)
    #print ('F1 Score : ' +str(f1))
    
    return acc_score, prec_score, recall

<h2>Separação entre treino e teste</h2>

In [99]:

# utiliza 25% do dataset para teste
trainData, validationData, trainLabels, validationLabels = train_test_split(data, labels, 
                                                    train_size=0.8, 
                                                    test_size=0.2, 
                                                    stratify=labels)

print(len(trainData), len(trainLabels))
print(len(validationData), len(validationLabels))

166 166
42 42


In [103]:
scores = []
scores_test = []



# Define quantos folds
kf = KFold(n_splits=10)

alpha = 0.5

print()
print("....Iniciando treinamento com 10 K-folds...." )
print()

kfold = 0    
for train_index, test_index in kf.split(trainData):
    
    print("################################################")
    print("K-fold : "+str(kfold+1))    
    print("################################################")
    #print(train_index, test_index)
    print()

    # Obten os subdados de treinamento e teste no n fold
    #---------------------------------------------------------------------
    X_train, X_test = trainData.iloc[train_index,:], trainData.iloc[test_index,:]
    #print(len(X_train), len(X_test))

    y_train, y_test = trainLabels.iloc[train_index], trainLabels.iloc[test_index]
    #print(len(y_train), len(y_test))
    
    print("....Inicializando vetor de pesos....")
    print()


    n_train, n_test = len(X_train), len(X_test)
    #pred_train, pred_test = [np.zeros(n_train), np.zeros(n_test)]

    # Initialize weights
    w = np.ones(n_train) / n_train
    print(w)
        
    # Fit um classificador
    clf_tree = DecisionTreeClassifier(max_depth = 1, random_state = 1)
    # Fit um classificador
    clf_rdf = RandomForestClassifier(max_depth=10)
    
    clf_svc = DecisionTreeClassifier()
    
    models = [clf_tree]

    model_index = 0
    for model in models:
        print()
        # Treina o modelo de classificação
        #---------------------------------------------------------------------
        print("Treinando o modelo....")

        # Treina o classificador com os pesos de treinamento
        model.fit(X=X_train, y=y_train, sample_weight=w)
        print(model)

        # Classifica o treino
        pred_train_i = model.predict(X_train)
        #print(pred_train_i)

        # Classifica o teste
        pred_test_i = model.predict(X_test)
        #print(pred_test_i)        

        print()
        print("...:::: Avaliação ::::....  ")
        print()

        # Obtem o index dos erros da classificação de treino e teste
        #---------------------------------------------------------------------
        missTrain = [int(x) for x in (pred_train_i != y_train)]
        print("Training Miss : "+str(missTrain))
        missTest = [int(x) for x in (pred_test_i != y_test)]
        #print("Testing Miss : "+str(missTest))

        # Equivale os valores entre 1/-1 para atualização dos pesos
        #---------------------------------------------------------------------
        miss2Train = [x if x==1 else -1 for x in missTrain]
        #print("Training Miss2 : "+str(miss2))
        miss2Test = [x if x==1 else -1 for x in missTest]
        #print("Testing Miss2 : "+str(miss2Test))


        # Calcula o erro
        #---------------------------------------------------------------------
        err_m = np.dot(w, missTrain) / sum(w)
        print("Error : "+str(err_m))

        # Calcula o Alpha 
        #---------------------------------------------------------------------
        alpha_m = alpha * np.log( (1 - err_m) / float(err_m))
        print("Alpha : "+str(alpha_m))


        # Mostra a Matriz de Confusão para treino e teste
        #---------------------------------------------------------------------
        print()
        print(":: Treinamento :: ")
        print("")
        train_acc_score, train_precision_score, train_recall_score = printCM(y_train, pred_train_i)

        #print()
        print(":: Teste ::")
        print()
        test_acc_score, test_precision_score, test_recall_score = printCM(y_test, pred_test_i)
        print


        # Atualiza os valores dos pesos
        #---------------------------------------------------------------------
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2Train]))
        print()
        print("Novos pesos atualizados : ")
        print(w)
        print()
        print("---------------------------------------------------------------------------")
        print()


        scores.append([kfold, model_index, train_acc_score, train_precision_score, train_recall_score, err_m, alpha_m, model])
        scores_test.append([kfold, model_index, test_acc_score, test_precision_score, test_recall_score, err_m, alpha_m, model])
        model_index += 1

    kfold += 1 
    print
    print


....Iniciando treinamento com 10 K-folds....

################################################
K-fold : 1
################################################

....Inicializando vetor de pesos....

[0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141 0.00671141
 

/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


Confusion Matrix : 
[[4 2]
 [6 5]]
Total : 208.0
Acurácia : 0.5294117647058824
Misclassification rate : 0.038
True positives : 0.455
False positives : 0.333
Specificity : 0.667
Precision : 0.6033613445378152
Prevalence : 0.053
Recall : 0.455

Novos pesos atualizados : 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]

---------------------------------------------------------------------------

################################################
K-fold : 4
################################################

....Inicializando vetor de pesos....

[0.00671141 0.00671141 0.00671141 0.00671141 0.00

/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in double_scalars
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

...:::: Avaliação ::::....  

Training Miss : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Error : 0.019016227180527343
Alpha : 1.9716316205821023

:: Treinamento :: 

Confus

/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

...:::: Avaliação ::::....  

Training Miss : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Error : 0.01428571428571432
Alpha : 2.1170532522986285

:: Treinamento :: 

Confusi

/Users/roberto/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


<h2>Apresentação dos Resultados</h2>

In [101]:
scoresDF = pd.DataFrame(scores, columns=["n-fold", 
                                         "Model Idx",
                                         "Acc", 
                                         "Precision", 
                                         "Recall",
                                         "Error", 
                                         "Alpha", 
                                         "Model"])

scoresDF.sort_values(["Acc","Error"], ascending=False)

,n-fold,Model Idx,Acc,Precision,Recall,Error,Alpha,Model
2,0,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."
5,1,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."
8,2,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."
11,3,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."
13,4,1,1.000000,1.000000,1.000,0.000000,inf,"(DecisionTreeClassifier(class_weight=None, cri..."
16,5,1,1.000000,1.000000,1.000,0.000000,inf,"(DecisionTreeClassifier(class_weight=None, cri..."
20,6,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."
23,7,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."
26,8,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."
29,9,2,1.000000,1.000000,1.000,0.000000,inf,"DecisionTreeClassifier(class_weight=None, crit..."


In [89]:
bestModel = scoresDF.iloc[0, 7]

In [90]:
pred_validation = bestModel.predict(validationData)

In [59]:
printCM(validationLabels, pred_validation)

Confusion Matrix : 
[[19  9]
 [13 11]]

Total : 208.0
Acurácia : 0.576923076923
Misclassification rate : 0.106
True positives : 0.458
False positives : 0.321
Specificity : 0.679
Precision : 0.55
Prevalence : 0.115
Recall : 0.458


(0.5769230769230769, 0.55, 0.458)

In [77]:
scoresDF = pd.DataFrame(scores_test, columns=["n-fold", 
                                         "Model Idx",
                                         "Acc", 
                                         "Precision", 
                                         "Recall",
                                         "Error", 
                                         "Alpha", 
                                         "Model"])

scoresDF.sort_values(["Acc","Error"], ascending=False)

,n-fold,Model Idx,Acc,Precision,Recall,Error,Alpha,Model
15,7,1,0.937500,1.000,0.900,0.000000,inf,"(DecisionTreeClassifier(class_weight=None, cri..."
13,6,1,0.875000,1.000,0.600,0.008696,2.368099,"(DecisionTreeClassifier(class_weight=None, cri..."
0,0,0,0.823529,1.000,0.571,0.248322,0.553790,"DecisionTreeClassifier(class_weight=None, crit..."
1,0,1,0.823529,0.833,0.714,0.040541,1.582034,"(DecisionTreeClassifier(class_weight=None, cri..."
7,3,1,0.823529,0.700,1.000,0.004310,2.721209,"(DecisionTreeClassifier(class_weight=None, cri..."
19,9,1,0.812500,1.000,0.667,0.015152,2.087194,"(DecisionTreeClassifier(class_weight=None, cri..."
11,5,1,0.764706,0.857,0.667,0.014286,2.117053,"(DecisionTreeClassifier(class_weight=None, cri..."
5,2,1,0.764706,1.000,0.556,0.004310,2.721209,"(DecisionTreeClassifier(class_weight=None, cri..."
9,4,1,0.764706,0.714,0.714,0.000000,inf,"(DecisionTreeClassifier(class_weight=None, cri..."
16,8,0,0.750000,0.667,0.667,0.240000,0.576340,"DecisionTreeClassifier(class_weight=None, crit..."
